##Wikipedia API

#To Do

1. How do get the concepts from Concept Insights?

2. Do we need to train to answer question #1?

3. If so, how do we train?

4. If we don't need to train, go back to question #1?

In [1]:
from wikipedia import Wikipedia
from wiki2plain import Wiki2Plain
import numpy as np

In [2]:
import ast

In [3]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )

In [ ]:
import os.path

def scrape_wiki(path,topic):
    '''
    Get all text from wiki page based on the topic entered.
    '''
    lang = 'en' # standard english wiki pages
    wiki = Wikipedia(lang)
    
    fname = path + topic + '.txt'
    
    if not os.path.isfile(fname):    
        try:
            raw = wiki.article(s)
        except:
            raw = None

        if raw:
            wiki2plain = Wiki2Plain(raw)
            content = wiki2plain.text

            with open(fname,'w') as handle:
                handle.write(content) 

path = "data/"
 

In [24]:
           
subjects = ["Calculus", 
            "Trigonometry", 
            "Geometry",
            "Differential Equations", 
            "Integrals", 
            "Unit Circle", 
            "Sine", 
            "Cosine", 
            "pythagorean theorem"]

In [4]:
for subject in subjects:
    scrape_wiki(path,subject)

In [5]:
f = open(path + subjects[-1] + '.txt','r')

In [6]:
print(f.read())

In mathematics, the Pythagorean theorem, also known as Pythagoras' theorem, is a relation in Euclidean geometry among the three sides of a right triangle. It states that the square of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the other two sides. The theorem can be written as an equation relating the lengths of the sides a, b and c, often called the "Pythagorean equation":
:a^2 + b^2 = c^2 ,
where c represents the length of the hypotenuse and a and b the lengths of the triangle's other two sides.

Although it is often argued that knowledge of the theorem predates him, the theorem is named after the ancient Greek mathematician Pythagoras ( 570 –  495 BC) as it is he who, by tradition, is credited with its first recorded proof.<!--
 --> <!--
 -->There is some evidence that Babylonian mathematicians understood the formula, although little of it indicates an application within a mathematical framework.<!--
 --><!--
 --> <!--
 -->Mesopotamian, 

##Concept Insights API

##Build Corpus

In [7]:
import json
import requests

In [9]:
# the ConceptInsights api from watson_developer_cloud is difficult to work with
# so I've been using request instead
from watson_developer_cloud import ConceptInsightsV2 as ConceptInsights
from pprint import pprint

In [8]:
# get account_id
base_url = "https://watson-api-explorer.mybluemix.net/concept-insights/api"
account_url = 'https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/accounts'
auth=('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52','50KJG6jdgZ6h')

r = requests.get(account_url, auth=auth)

In [9]:
# display account id
print(r.content)

{"accounts":[{"account_id":"liet4blerinu2"}]}


In [56]:
# create copus called 'math'
# I had a very hard time trying to get this code to work - hours lost ! 
# It still doesn't work. Maybe I should be using '.post' instead of '.put' ...?

# create_corp = "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math"
# payload = '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'
# r2 = requests.put(create_corp,
#                   params = payload,
#                   auth=auth)
# r2.status_code

400

In [10]:
# create corpus called 'math_corp'
# this code only needs to run once -- otherwise 409 error will occur

!curl -D - -X PUT -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
    -d  '{"access":"private","users":[{"uid":"82a698f0-8d8c-42f9-9c7d-1e4b97eebd52","permission":"AdminReadWrite"}]}'\
    "https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp"














{"error":"corpus already exists","url":"/api/v2/corpora/liet4blerinu2/math_corp"}


In [11]:
# get available corpora
available_corpora = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2'

r = requests.get(available_corpora, auth=auth)
r.status_code

200

In [12]:
# display available corpora and it's parameters
ast.literal_eval(r.content)

{'corpora': [{'access': 'private',
   'id': '/corpora/liet4blerinu2/math_corp',
   'users': [{'account_id': '', 'permission': 'ReadWriteAdmin'},
    {'account_id': 'liet4blerinu2', 'permission': 'ReadWriteAdmin'}]},
  {'access': 'private',
   'id': '/corpora/liet4blerinu2/math_corp2',
   'users': [{'account_id': '', 'permission': 'ReadWriteAdmin'},
    {'account_id': 'liet4blerinu2', 'permission': 'ReadWriteAdmin'}]}]}

In [14]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/math_corp2/liet4blerinu2 /math_corp2'
requests.delete(url)

<Response [401]>

In [153]:
# trying to use Request to upload file from disk, so far no success
# subject = "Calculus.txt"


# url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/Calculus'
# payload = {
#           "label": 'Calculus',
#           "parts": [
#           {
#              "data": open(path + subject,'rb').read(),
#              "name": "HTML part",
#              "content-type": "text/html"
#           }]}
# r2 = requests.post(url,
#                   params=payload,
#                   auth=auth)
# r2.status_code                 

400

In [148]:
# trying to use Request to upload file, so far no success
# files = {'upload_file': open(path + subject,'rb')}

# r2 = requests.post(url,
#                   files=files,
#                   auth=auth)
# r2.status_code

400

In [67]:
# create and upload file into corpus maath_corp
! curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h'\
    -X PUT 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument' \
    -d '{"label": "Test document","parts": [{"data": "<div><h2>Part of test document using HTML</h2></div>","name": "HTML part","content-type": "text/html"},{"data": "Part of test document pure text.","name": "Text part","content-type": "text/plain"}],"user_fields": {"myField1": "my meta-data"}}'

In [48]:
# can upload file in request, provided file is manually typed string...
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'
path ="data/"
for subject in subjects:
    url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject

    payload = {"label": subject,
               "parts": [{"data": open(path + subject + '.txt','rb').read(),
                          "name": "HTML part",
                          "content-type": "text/html"},
                         {"data": "Part of test document pure text."
                          ,"name": "Text part",
                          "content-type": "text/plain"}],
               "user_fields": {"myField1": "my meta-data"}}

    r2 = requests.post(url = url,
                      data = json.dumps(payload),
                      auth=auth)
    print(r2.status_code)

201
201
201
201
201
201
201
201
201


In [ ]:
 open(path + subjects[] + '.txt','rb').read()

In [49]:
# retrive document id's
# GET /v2/corpora/{account_id}/{corpus}/documents
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents'
r = requests.get(url,auth=auth)

print (r.status_code)

print (ast.literal_eval(r.content))

200
{'documents': ['/corpora/liet4blerinu2/math_corp/documents/testDocument', '/corpora/liet4blerinu2/math_corp/documents/Calculus', '/corpora/liet4blerinu2/math_corp/documents/Trigonometry', '/corpora/liet4blerinu2/math_corp/documents/Geometry', '/corpora/liet4blerinu2/math_corp/documents/Differential%20Equations', '/corpora/liet4blerinu2/math_corp/documents/Integrals', '/corpora/liet4blerinu2/math_corp/documents/Unit%20Circle', '/corpora/liet4blerinu2/math_corp/documents/Sine', '/corpora/liet4blerinu2/math_corp/documents/Cosine', '/corpora/liet4blerinu2/math_corp/documents/pythagorean%20theorem']}


In [50]:
# get content of document
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'id': '/corpora/liet4blerinu2/math_corp/documents/testDocument',
 'label': 'Calculus',
 'last_modified': '2015-12-01T23:34:46.216Z',
 'parts': [{'content-type': 'text/html',
   'data': 'Calculus is the mathematical study of change, in the same way that geometry is the study of shape and algebra is the study of operations and their application to solving equations. It has two major branches, differential calculus (concerning rates of change and slopes of curves), and integral calculus (concerning accumulation of quantities and the areas under and between curves); these two branches are related to each other by the fundamental theorem of calculus. Both branches make use of the fundamental notions of convergence of infinite sequences and infinite series to a well-defined limit. Generally, modern calculus is considered to have been developed in the 17th century by Isaac Newton and Gottfried Leibniz. Today, calculus has widespread uses in science, engineering and economics and can solve ma

In [32]:
# delete document from corpus by id
# DELETE /v2/corpora/{account_id}/{corpus}/documents/{document}

! curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h'\
       -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'

In [53]:
# GET /v2/corpora/{account_id}/{corpus}/stats
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/stats'
r = requests.get(url,auth=auth)
r.status_code

200

In [54]:
# stats on concepts present in corpora 
ast.literal_eval(r.content)

{'id': '/corpora/liet4blerinu2/math_corp',
 'last_updated': '2015-12-01T23:41:39.572Z',
 'top_tags': {'corpus_tags_histogram': {'1': 742,
   '10': 11,
   '101': 1,
   '11': 16,
   '114': 1,
   '118': 1,
   '12': 19,
   '122': 1,
   '13': 12,
   '135': 1,
   '14': 5,
   '148': 1,
   '15': 9,
   '153': 1,
   '154': 1,
   '16': 9,
   '160': 1,
   '17': 3,
   '173': 1,
   '18': 5,
   '182': 1,
   '19': 2,
   '195': 1,
   '2': 362,
   '20': 8,
   '203': 1,
   '21': 1,
   '22': 5,
   '226': 1,
   '23': 1,
   '236': 1,
   '24': 5,
   '25': 2,
   '26': 1,
   '27': 1,
   '28': 2,
   '29': 2,
   '3': 120,
   '30': 2,
   '31': 2,
   '32': 2,
   '33': 2,
   '338': 1,
   '34': 2,
   '35': 2,
   '39': 1,
   '4': 95,
   '40': 2,
   '42': 1,
   '428': 1,
   '45': 3,
   '46': 2,
   '47': 2,
   '49': 2,
   '5': 39,
   '50': 1,
   '52': 2,
   '53': 2,
   '54': 1,
   '55': 1,
   '57': 2,
   '58': 1,
   '6': 33,
   '60': 1,
   '65': 2,
   '68': 1,
   '7': 33,
   '8': 20,
   '9': 21,
   '92': 1,
   '93': 1,

In [71]:
# GET /v2/corpora/{account_id}/{corpus}/label_search
# GET /v2/corpora/{account_id}/{corpus}/conceptual_search
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/conceptual_search?query=Calculus'
r = requests.get(url,auth=auth)
r.status_code

400

In [68]:
ast.literal_eval(r.content)

{'matches': []}

In [73]:
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/graphs'
r = requests.get(url,auth=auth)
r.status_code

200

In [74]:
ast.literal_eval(r.content)

{'graphs': ['/graphs/wikipedia/en-20120601', '/graphs/wikipedia/en-latest']}

##ConceptExpansion API

In [7]:
import json
import time
from watson_developer_cloud import ConceptExpansionV1Beta as ConceptExpansion


concept_expansion = ConceptExpansion(username='82a698f0-8d8c-42f9-9c7d-1e4b97eebd52',
                                     password='50KJG6jdgZ6h')

job_id = concept_expansion.create_job(dataset='mtsamples', 
                                      seeds=['motrin', 'tylenol', 'aspirin'], 
                                      label='medications')
print(json.dumps(job_id, indent=2))

time.sleep(5)  # sleep for 5 seconds
job_status = concept_expansion.get_status(job_id)

while job_status['status'] == 'in progress' or job_status['status'] == 'awaiting work':
    time.sleep(5)  # sleep for 5 seconds
    job_status = concept_expansion.get_status(job_id)
    print(json.dumps(job_status, indent=2))

if job_status['status'] == 'done':
    results = concept_expansion.get_results(job_id)
    print(json.dumps(results, indent=2))

{
  "jobid": "M0hUQkdKOTZJWkNDSUhRSVVCT047MTI0MzAxMTE="
}
{
  "return_seeds": [
    {
      "result": "ibuprofen", 
      "prevalence": 0.8219226002693176
    }, 
    {
      "result": "advil", 
      "prevalence": 0.80472731590271
    }, 
    {
      "result": "naproxen", 
      "prevalence": 0.7841069102287292
    }, 
    {
      "result": "benadryl", 
      "prevalence": 0.7685651183128357
    }, 
    {
      "result": "acetaminophen", 
      "prevalence": 0.7564734220504761
    }, 
    {
      "result": "codeine", 
      "prevalence": 0.743308424949646
    }, 
    {
      "result": "pain relievers", 
      "prevalence": 0.7278230786323547
    }, 
    {
      "result": "laxatives", 
      "prevalence": 0.7256090641021729
    }, 
    {
      "result": "motrin ib", 
      "prevalence": 0.7229762077331543
    }, 
    {
      "result": "aleve", 
      "prevalence": 0.7223989367485046
    }, 
    {
      "result": "diuretics", 
      "prevalence": 0.7221292853355408
    }, 
    {
      "